# Modelling Cu-MOF-74: H2O adsorption with DMET

We model the $H_2O @ Cu-MOF-74$ by fragmenting the MOF into interacting fragments. The system consists of 57 atoms. We decompose the system 54 fragments. Only one fragment has 4 atoms ($Cu$, $H_2O$) and all the remaining atoms (53) are 53 fragments. Classical electronic solver CCSD is used for all the fragment. We perfrom a 1-shot version of DMET.


![cu-mof](../images/cu-mof/cu-mof.png)

Since quantum computers are limited in computational spac, choosing only $Cu-H_2O$ fragment for modelling with quantum computing was expensive with DMET due to high number of bath orbitals constructed and high number of hamiltonian terms/qubits even after active space reduction.

We choose the minimal basis set STO-3G due to limited computing power and calculate the ground state energy at stable configuration obtained from quantistry. We compare the results obtained from Restricted Hartree fock (RHF) method.

In [75]:
import sys
sys.path.append('..')

In [76]:
import numpy as np
import py3Dmol
from dmet.dmet_problem_decomposition_single import DMETProblemDecompositionSingle
from dmet.meta_lowdin_localization import meta_lowdin_localization
from dmet.ccsd_solver import CCSDSolver
from pyscf import gto
from utils import rel_err
import pickle
from pyscf import scf
import matplotlib.pyplot as plt
import pandas as pd
import time

In [87]:
symbols = []
coordinates = []

file = 'cu-mof74-h2o.xyz'
with open(file, encoding ='utf-8') as f:
    for line in f.readlines()[2:]:
        symbol, x, y, z, _ = line.split()
        symbols.append(symbol)
        coordinates.append(float(x))
        coordinates.append(float(y))
        coordinates.append(float(z))

In [88]:
geometry = ""
for index, symbol in enumerate(symbols):
    geometry += f"{symbol} {coordinates[3*index]} {coordinates[3*index+1]} {coordinates[3*index+2]} ; "
geometry = geometry[:-2]

In [89]:
temp_g = geometry.split(';')
temp_g

['Cu 0.05033917 2.8950158 3.09316041 ',
 ' Mg 2.10073752 4.35951009 9.93744675 ',
 ' Mg 4.75529817 4.91186968 2.18929815 ',
 ' Mg -2.38118808 2.61665623 10.69016152 ',
 ' Mg 2.3284059 3.46840696 1.14056334 ',
 ' Mg -0.13400514 4.13331338 11.8344551 ',
 ' H 2.34965049 -0.12315283 0.81738382 ',
 ' H -0.13026902 7.5984989 12.24149511 ',
 ' H 0.14832424 4.45801951 6.18637226 ',
 ' H 2.12136951 3.02017799 6.69980208 ',
 ' H -0.01574471 8.30331089 0.44687104 ',
 ' H 2.36694014 -0.71904433 12.49615912 ',
 ' C 5.86909309 1.07219866 0.4588129 ',
 ' C -3.64431934 6.41684944 12.57249031 ',
 ' C -1.38715448 3.85237208 7.58233733 ',
 ' C 3.65108472 3.65628224 5.29977876 ',
 ' C 1.48645793 6.75696455 0.60072476 ',
 ' C 0.85585748 0.82862013 12.30674108 ',
 ' C 0.51572294 1.01907917 0.91814677 ',
 ' C 1.71772416 6.47305557 12.10771419 ',
 ' C -1.69452486 4.06975189 5.12705128 ',
 ' C 3.96938465 3.42635423 7.75732522 ',
 ' C 2.8504647 6.36981965 0.35291547 ',
 ' C -0.50512469 1.21114478 12.57260901 ',

In [90]:
len(temp_g)

57

In [91]:
copper = temp_g.pop(0)
temp_g.insert(-3, copper)

In [92]:
temp_g = temp_g[-4:] + temp_g[:-4]

In [93]:
geometry = ';'.join(temp_g)
geometry

'Cu 0.05033917 2.8950158 3.09316041 ; H -1.90488736 1.63572698 4.7203535 ; O -1.40151681 1.24589039 3.96234193 ; H -1.02296789 0.41555443 4.33033117 ; Mg 2.10073752 4.35951009 9.93744675 ; Mg 4.75529817 4.91186968 2.18929815 ; Mg -2.38118808 2.61665623 10.69016152 ; Mg 2.3284059 3.46840696 1.14056334 ; Mg -0.13400514 4.13331338 11.8344551 ; H 2.34965049 -0.12315283 0.81738382 ; H -0.13026902 7.5984989 12.24149511 ; H 0.14832424 4.45801951 6.18637226 ; H 2.12136951 3.02017799 6.69980208 ; H -0.01574471 8.30331089 0.44687104 ; H 2.36694014 -0.71904433 12.49615912 ; C 5.86909309 1.07219866 0.4588129 ; C -3.64431934 6.41684944 12.57249031 ; C -1.38715448 3.85237208 7.58233733 ; C 3.65108472 3.65628224 5.29977876 ; C 1.48645793 6.75696455 0.60072476 ; C 0.85585748 0.82862013 12.30674108 ; C 0.51572294 1.01907917 0.91814677 ; C 1.71772416 6.47305557 12.10771419 ; C -1.69452486 4.06975189 5.12705128 ; C 3.96938465 3.42635423 7.75732522 ; C 2.8504647 6.36981965 0.35291547 ; C -0.50512469 1.211

In [96]:
frag_1_atoms = [i.split()[0] for i in temp_g[:4]]
frag_2_atoms = [i.split()[0] for i in temp_g[4:]]

set(frag_1_atoms + frag_2_atoms)

{'C', 'Cu', 'H', 'Mg', 'O'}

In [98]:
len(frag_1_atoms), len(frag_2_atoms)

(4, 53)

In [100]:
mol = gto.Mole() # Instantiate the molecule class in PySCF
mol.atom = geometry
mol.basis = "sto-3g" # Use "minao" as the basis set
mol.charge = 1 # Assign the charge of the molecule
mol.spin = 0  # Assign the spin of the molecule
mol.build() # Build the molecule object

In [101]:
isum = 0
isum2 = -1

n_frag_freeze = []

for index, i in enumerate([4] + [1]*53 ): 
    itemp = 0
    isum2 += i
    freezer = []
    
    for total_basis in mol.spheric_labels():
        item = total_basis.split()
        item0 = int(item[0])
        if ( ( item0 >= isum ) and ( item0 <= isum2 ) ):
            orbital = int(item[2][0])
            shell = item[2][1]
            
            if (item[1] == 'C') or (item[1] == 'O'):
                if orbital != 2:
                    freezer.append(itemp)
            elif item[1] == 'Mg':
                if orbital != 3:
                    freezer.append(itemp)
                if shell != 's':
                    freezer.append(itemp)
            elif item[1] == 'Cu':
                if (orbital != 3) and (orbital != 4):
                    freezer.append(itemp)
                if (orbital == 4) and shell != 's':
                    freezer.append(itemp)
            itemp+=1
    
    if len(freezer) == 0:
        freezer = None
        n_frag_freeze.append(freezer)
    else:
        n_frag_freeze.append(freezer.copy())
        
    isum += i 

In [102]:
n_frag_freeze

[[0, 1, 4, 5, 6, 10, 11, 12, 19],
 [0, 1, 3, 3, 4, 4, 5, 5, 6, 7, 8],
 [0, 1, 3, 3, 4, 4, 5, 5, 6, 7, 8],
 [0, 1, 3, 3, 4, 4, 5, 5, 6, 7, 8],
 [0, 1, 3, 3, 4, 4, 5, 5, 6, 7, 8],
 [0, 1, 3, 3, 4, 4, 5, 5, 6, 7, 8],
 None,
 None,
 None,
 None,
 None,
 None,
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]]

In [104]:
dmet_solver = DMETProblemDecompositionSingle()
dmet_solver.frozen = n_frag_freeze

dmet_solver.electronic_structure_solver = CCSDSolver()
dmet_solver.electron_localization_method = meta_lowdin_localization

In [105]:
s = time.time()
energy = dmet_solver.simulate(mol, [4] + [1]*53)

e = time.time()
print("Duration:", e-s)

Active space:  range(0, 286)
Time for two-electron integrals(low_scf_twoint): 14.131298303604126
Dmet active orbitals: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Dmet active orbitals: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

 	Iteration =  1
 	----------------
 
		Fragment Number : #  1
		------------------------
		t list:  [25]

Time for dmet_onerdm_embed:  4.673004150390625e-05
Time for dmet_bath_orb_sort:  0.00017881393432617188
Time for dmet_add_to_bath_orb:  0.002207517623901367
		Bath Orbitals Constructed (286, 286)
		e_occupied (286,)
		t list after fragmet bath:  [25, 25]

t_list:  [25, 25]
		One Particle RDM Constructed (286, 286)
		Number of Electrons in Fragment 50
		Number of Orbitals in Fragment 50

Time for two-electron integrals(two_int): 13.805773973464966
Time for frag_twoint: 25.83957839012146
		One Particle Hamiltonian Constructed (50, 50)
		Fock Matrix Constructed (50, 50)
		Two Particle Hamiltonian Constructed (50, 50, 50, 50)

		Guess Orbitals Constructed (50, 50)

Time for SCF RHF object:  0.00014901161193847656
Time for ao2mo.restore:  0.0021905899047851562
Time for SCF:  0.12474322319030762
		SCF Calculation Done -1882.8061457496483
		Fock Matrix Constructed (50, 50)
		Molecule fra

Time for two-electron integrals(two_int): 14.023331880569458
Time for frag_twoint: 21.751338720321655
		One Particle Hamiltonian Constructed (18, 18)
		Fock Matrix Constructed (18, 18)
		Two Particle Hamiltonian Constructed (18, 18, 18, 18)

		Guess Orbitals Constructed (18, 18)

Time for SCF RHF object:  0.0001537799835205078
Time for ao2mo.restore:  7.367134094238281e-05
Time for SCF:  0.010651588439941406
		SCF Calculation Done -225.64496504054944
		Fock Matrix Constructed (18, 18)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7febdc152370>

Number of orbitals:  18
Frozen orbitals:  [0, 1, 3, 3, 4, 4, 5, 5, 6, 7, 8]
CCSD time:  0.015644550323486328
		Electronic Structure Solver Done -225.6463872881562

Lambda time:  0.02085089683532715
1-RDM time:  0.0001499652862548828
2-RDM time:  0.0146636962890625
(18, 18) (18, 18, 18, 18)
		CC One RDM Constructed (18, 18)
		CC Two RDM Constructed (18, 18, 18, 18)

		Fragment Energy Computed -414.7491845422709
		Total Energy R

Time for two-electron integrals(two_int): 13.969514846801758
Time for frag_twoint: 20.485701084136963
		One Particle Hamiltonian Constructed (2, 2)
		Fock Matrix Constructed (2, 2)
		Two Particle Hamiltonian Constructed (2, 2, 2, 2)

		Guess Orbitals Constructed (2, 2)

Time for SCF RHF object:  0.00014090538024902344
Time for ao2mo.restore:  4.863739013671875e-05
Time for SCF:  0.011718511581420898
		SCF Calculation Done -1.9691185107283058
		Fock Matrix Constructed (2, 2)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7febd7f5c850>

Number of orbitals:  2
Frozen orbitals:  None
CCSD time:  0.022081613540649414
		Electronic Structure Solver Done -1.983849705003466

Lambda time:  0.010135173797607422
1-RDM time:  0.00011110305786132812
2-RDM time:  0.0046308040618896484
(2, 2) (2, 2, 2, 2)
		CC One RDM Constructed (2, 2)
		CC Two RDM Constructed (2, 2, 2, 2)

		Fragment Energy Computed -15.508871863518554
		Total Energy RDM Computed -1.983849704944593
		One RDM Comput

Time for dmet_bath_orb_sort:  0.00020956993103027344
Time for dmet_add_to_bath_orb:  0.0008840560913085938
		Bath Orbitals Constructed (286, 286)
		e_occupied (286,)
		t list after fragmet bath:  [5, 5]

t_list:  [5, 5]
		One Particle RDM Constructed (286, 286)
		Number of Electrons in Fragment 10
		Number of Orbitals in Fragment 10

Time for two-electron integrals(two_int): 14.105265378952026
Time for frag_twoint: 21.15879797935486
		One Particle Hamiltonian Constructed (10, 10)
		Fock Matrix Constructed (10, 10)
		Two Particle Hamiltonian Constructed (10, 10, 10, 10)

		Guess Orbitals Constructed (10, 10)

Time for SCF RHF object:  0.0001423358917236328
Time for ao2mo.restore:  5.650520324707031e-05
Time for SCF:  0.010905981063842773
		SCF Calculation Done -53.34747488443845
		Fock Matrix Constructed (10, 10)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7febf9386640>

Number of orbitals:  10
Frozen orbitals:  [0]
CCSD time:  0.03902745246887207
		Electronic Struc

...

Lambda time:  0.04358410835266113
1-RDM time:  0.00018906593322753906
2-RDM time:  0.0070955753326416016
(10, 10) (10, 10, 10, 10)
		CC One RDM Constructed (10, 10)
		CC Two RDM Constructed (10, 10, 10, 10)

		Fragment Energy Computed -169.50815472536988
		Total Energy RDM Computed -54.43863947091895
		One RDM Computed (10, 10)

		Fragment Energy                 =   -169.5081547254
		Number of Electrons in Fragment =     10.0000000000

		Fragment Number : #  29
		------------------------
		t list:  [5]

Time for dmet_onerdm_embed:  0.00018358230590820312
Time for dmet_bath_orb_sort:  0.00017189979553222656
Time for dmet_add_to_bath_orb:  0.0007252693176269531
		Bath Orbitals Constructed (286, 286)
		e_occupied (286,)
		t list after fragmet bath:  [5, 5]

t_list:  [5, 5]
		One Particle RDM Constructed (286, 286)
		Number of Electrons in Fragment 10
		Number of Orbitals in Fragment 10

Time for two-electron integrals(two_int): 13.837964296340942
Time for frag_twoint: 20.61782455444336
		

Time for dmet_bath_orb_sort:  0.00030303001403808594
Time for dmet_add_to_bath_orb:  0.001069784164428711
		Bath Orbitals Constructed (286, 286)
		e_occupied (286,)
		t list after fragmet bath:  [5, 5]

t_list:  [5, 5]
		One Particle RDM Constructed (286, 286)
		Number of Electrons in Fragment 10
		Number of Orbitals in Fragment 10

Time for two-electron integrals(two_int): 13.849609375
Time for frag_twoint: 20.572185516357422
		One Particle Hamiltonian Constructed (10, 10)
		Fock Matrix Constructed (10, 10)
		Two Particle Hamiltonian Constructed (10, 10, 10, 10)

		Guess Orbitals Constructed (10, 10)

Time for SCF RHF object:  0.0001506805419921875
Time for ao2mo.restore:  5.245208740234375e-05
Time for SCF:  0.009409666061401367
		SCF Calculation Done -55.78047763582789
		Fock Matrix Constructed (10, 10)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7febe484f640>

Number of orbitals:  10
Frozen orbitals:  [0]
CCSD time:  0.030544519424438477
		Electronic Structure 

Time for two-electron integrals(two_int): 13.836760997772217
Time for frag_twoint: 20.55594801902771
		One Particle Hamiltonian Constructed (10, 10)
		Fock Matrix Constructed (10, 10)
		Two Particle Hamiltonian Constructed (10, 10, 10, 10)

		Guess Orbitals Constructed (10, 10)

Time for SCF RHF object:  0.000148773193359375
Time for ao2mo.restore:  5.1975250244140625e-05
Time for SCF:  0.010723352432250977
		SCF Calculation Done -85.1471581333922
		Fock Matrix Constructed (10, 10)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7febe7f62d90>

Number of orbitals:  10
Frozen orbitals:  [0]
CCSD time:  0.046570539474487305
		Electronic Structure Solver Done -85.21887074130409

Lambda time:  0.04750657081604004
1-RDM time:  0.00039458274841308594
2-RDM time:  0.008083820343017578
(10, 10) (10, 10, 10, 10)
		CC One RDM Constructed (10, 10)
		CC Two RDM Constructed (10, 10, 10, 10)

		Fragment Energy Computed -270.0256598659695
		Total Energy RDM Computed -85.2188707361424


Lambda time:  0.06807684898376465
1-RDM time:  0.00017499923706054688
2-RDM time:  0.0073108673095703125
(10, 10) (10, 10, 10, 10)
		CC One RDM Constructed (10, 10)
		CC Two RDM Constructed (10, 10, 10, 10)

		Fragment Energy Computed -224.69202256795333
		Total Energy RDM Computed -84.55354104001512
		One RDM Computed (10, 10)

		Fragment Energy                 =   -224.6920225680
		Number of Electrons in Fragment =     10.0000000000

		Fragment Number : #  45
		------------------------
		t list:  [5]

Time for dmet_onerdm_embed:  0.00018095970153808594
Time for dmet_bath_orb_sort:  0.0001862049102783203
Time for dmet_add_to_bath_orb:  0.0006868839263916016
		Bath Orbitals Constructed (286, 286)
		e_occupied (286,)
		t list after fragmet bath:  [5, 5]

t_list:  [5, 5]
		One Particle RDM Constructed (286, 286)
		Number of Electrons in Fragment 10
		Number of Orbitals in Fragment 10

Time for two-electron integrals(two_int): 13.82520866394043
Time for frag_twoint: 20.690864324569702
		O

Time for two-electron integrals(two_int): 13.90587592124939
Time for frag_twoint: 20.721293926239014
		One Particle Hamiltonian Constructed (10, 10)
		Fock Matrix Constructed (10, 10)
		Two Particle Hamiltonian Constructed (10, 10, 10, 10)

		Guess Orbitals Constructed (10, 10)

Time for SCF RHF object:  0.00013518333435058594
Time for ao2mo.restore:  5.3882598876953125e-05
Time for SCF:  0.012365579605102539
		SCF Calculation Done -83.94519343485123
		Fock Matrix Constructed (10, 10)
		Molecule fragment Constructed <pyscf.gto.mole.Mole object at 0x7febe48d5fa0>

Number of orbitals:  10
Frozen orbitals:  [0]
CCSD time:  0.04646587371826172
		Electronic Structure Solver Done -84.00718844655498

Lambda time:  0.051531314849853516
1-RDM time:  0.0003361701965332031
2-RDM time:  0.007877349853515625
(10, 10) (10, 10, 10, 10)
		CC One RDM Constructed (10, 10)
		CC Two RDM Constructed (10, 10, 10, 10)

		Fragment Energy Computed -246.65364645979383
		Total Energy RDM Computed -84.00718844321

In [107]:
energy

-4867.177019552769

In [111]:
actual_energy_rhf = -4909.639090862013

In [114]:
total_energy = energy
print(f'Actual total energy: {actual_energy_rhf:.12f} Ha\n')
print(f"Total energy: {total_energy:.12f} Ha")

rel_error = rel_err(actual_energy_rhf, total_energy)
print(f'Relative error: {rel_error:.12f}')

Actual total energy: -4909.639090862013 Ha

Total energy: -4867.177019552769 Ha
Relative error: 0.008648715419


## Results

The algorithm halted after running for ~2313 seconds (approx 40 minutes). The final DMET energy obtained is `4867.17702` Ha with `389.66` electrons (actual `392` electrons).

Compared to the energy value obtained from RHF method, the relative error is within $10^{-2}$ Ha. We tried to simulate the system with CCSD solver (with frozen orbitals) but it was intractable.

## Summary

In this notebook, we explored $H_2O @ Cu-MOF-74$ system using DMET and a simple fragmentation stratergy. We were able to run the alogrithm in feasible time and compared the results with RHF method. The results are promising and within $10^{-2}$ Ha relative error. In future, we can extend the method to use quantum computing method for high accuracy solvers.